**This code implements a deeep learning workflow for Real-Time Emotion Recognition System Using Facial Expressions.**

Install required libraries and modules

In [1]:
pip install matplotlib

In [2]:
pip install keras


Note: you may need to restart the kernel to use updated packages.


In [3]:
pip install tensorflow


Note: you may need to restart the kernel to use updated packages.


In [4]:
pip install seaborn

Note: you may need to restart the kernel to use updated packages.


In [5]:
pip install keras_preprocessing

Note: you may need to restart the kernel to use updated packages.


In [6]:
pip install tqdm

Note: you may need to restart the kernel to use updated packages.


To access the Kaggle Face Expression Recognition dataset in Google Colab without downloading it manually

In [7]:
!pip install kaggle --quiet


Authenticate to Kaggle (One-time setup):

Create a Kaggle API token: https://www.kaggle.com/docs/api


In [8]:
from google.colab import drive
drive.mount('/content/gdrive')



ModuleNotFoundError: No module named 'google.colab'

In [ ]:
!mkdir ~/.kaggle


In [ ]:
!cp /content/gdrive/MyDrive/kaggle.json ~/.kaggle/kaggle.json

Download dataset from kaggel

In [10]:
import os

# Specify dataset path 
dataset_name = 'jonathanoheix/face-expression-recognition-dataset'

# Download to temporary directory
download_path = os.path.join('/content', 'tmp')
!kaggle datasets download -p {download_path} -d {dataset_name}

# Unzip the downloaded file 
!unzip {os.path.join(download_path, os.listdir(download_path)[0])}


Dataset URL: https://www.kaggle.com/datasets/jonathanoheix/face-expression-recognition-dataset
License(s): unknown
face-expression-recognition-dataset.zip: Skipping, found more recently modified local copy (use --force to force download)


'unzip' is not recognized as an internal or external command,
operable program or batch file.


In [ ]:
!ls /content/images

**Importing Libraries and Modules**

In [ ]:
import tensorflow.keras as keras
from keras.utils import to_categorical
from keras_preprocessing.image import load_img
from keras.models import Sequential
from keras.layers import Dense, Conv2D, Dropout, Flatten, MaxPooling2D
import os
import pandas as pd
import numpy as np

**Data Preparation**

In [ ]:
current_dir='/content'
print(current_dir)
TRAIN_DIR = current_dir+'/images/train'
TEST_DIR = current_dir+'/images/validation'

This function creates a pandas DataFrame that essentially collects the file paths of images and their corresponding labels, organizing them for further processing.

In [ ]:
def createdataframe(dir):
    image_paths = []
    labels = []
    for label in os.listdir(dir):
        for imagename in os.listdir(os.path.join(dir, label)):
            image_paths.append(os.path.join(dir, label, imagename))
            labels.append(label)
        print(label, "completed")
    return image_paths, labels

The function returns the lists of image_paths and labels containing information about all the images in the given directory.

In [ ]:
train = pd.DataFrame()
train['image'], train['label'] = createdataframe(TRAIN_DIR)
print(train)

It creates the training DataFrame using the createdataframe function.

In [ ]:
test = pd.DataFrame()
test['image'], test['label'] = createdataframe(TEST_DIR)
print(test)

This line imports the tqdm module for displaying a progress bar.

In [ ]:
from tqdm.notebook import tqdm

This function reads and processes images using the tqdm progress bar.

In [ ]:
def extract_features(images):
    features = []
    for image in tqdm(images):
        img = load_img(image, grayscale=True)
        img = np.array(img)
        features.append(img)
    features = np.array(features)
    features = features.reshape(len(features), 48, 48, 1)
    return features

Extraction: The function returns the extracted features in the form of a NumPy array.

In [ ]:
train_features = extract_features(train['image'])
test_features = extract_features(test['image'])

Normalization:Each pixel value in the images (ranging from 0 to 255) is divided by 255.0.
This operation scales the pixel values to the range of 0 to 1.

In [ ]:
x_train = train_features / 255.0
x_test = test_features / 255.0

This part encodes the labels using LabelEncoder and converts them into categorical format.

In [ ]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
le.fit(train['label'])
y_train = le.transform(train['label'])
y_test = le.transform(test['label'])
y_train = to_categorical(y_train, num_classes=7)
y_test = to_categorical(y_test, num_classes=7)

Building the Neural Network Model

In [ ]:
# sequential model
model = Sequential()

# convolutional layers
model.add(Conv2D(128, kernel_size=(3,3), activation='relu', input_shape=(48,48,1)))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.4))

model.add(Conv2D(256, kernel_size=(3,3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.4))

model.add(Conv2D(512, kernel_size=(3,3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.4))

model.add(Conv2D(512, kernel_size=(3,3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.4))

# flattening
model.add(Flatten())

# fully connected layers
model.add(Dense(512, activation='relu'))
model.add(Dropout(0.4))
model.add(Dense(256, activation='relu'))
model.add(Dropout(0.3))

# output layer
model.add(Dense(7, activation='softmax'))

# model compilation
#model.compile(optimizer = 'adam', loss = 'categorical_crossentropy', metrics = 'accuracy' )
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

**Training the Model**

In [ ]:
model.fit(x=x_train, y=y_train, batch_size=128, epochs=100, validation_data=(x_test, y_test))

Saving the Model : It saves the trained model architecture and weights to files.

In [ ]:
model_json = model.to_json()
with open("emotiondetector.json", 'w') as json_file:
    json_file.write(model_json)
model.save("emotiondetector.h5")

Loading the Model and Making Predictions

In [ ]:
from keras.models import model_from_json
json_file = open("emotiondetector.json", "r")
model_json = json_file.read()
json_file.close()
model = model_from_json(model_json)
model.load_weights("emotiondetector.h5")

Making Predictions on Test Images

In [ ]:
label = ['angry', 'disgust', 'fear', 'happy', 'neutral', 'sad', 'surprise']
def ef(image):
    img = load_img(image, grayscale=True)
    feature = np.array(img)
    feature = feature.reshape(1, 48, 48, 1)
    return feature / 255.0

Test prediction

In [ ]:
image = current_dir+'/images/train/sad/42.jpg'
print("original image is of sad")
img = ef(image)
pred = model.predict(img)
pred_label = label[pred.argmax()]
print("model prediction is ", pred_label)

Visualizing Predictions

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline

**Test**

In [ ]:
image = current_dir+'/images/train/sad/42.jpg'
print("original image is of sad")
img = ef(image)
pred = model.predict(img)
pred_label = label[pred.argmax()]
print("model prediction is ", pred_label)
plt.imshow(img.reshape(48, 48), cmap='gray')

try with another image

In [ ]:
image = current_dir+ '/images/train/fear/2.jpg'
print("original image is of fear")
img = ef(image)
pred = model.predict(img)
pred_label = label[pred.argmax()]
print("model prediction is ",pred_label)
plt.imshow(img.reshape(48,48),cmap='gray')

**Making the model real time using OpenCV**

In [ ]:
import cv2
from keras.models import model_from_json
import numpy as np

# Load the pre-trained model architecture from JSON file
#json_file = open("facialemotionmodel.json", "r")
json_file = open("emotiondetector.json", "r")
model_json = json_file.read()
json_file.close()
model = model_from_json(model_json)

# Load the pre-trained model weights
#model.load_weights("facialemotionmodel.h5")
model.load_weights("emotiondetector.h5")

# Load the Haar cascade classifier for face detection
haar_file = cv2.data.haarcascades + 'haarcascade_frontalface_default.xml'
face_cascade = cv2.CascadeClassifier(haar_file)

# Define a function to extract features from an image
def extract_features(image):
    feature = np.array(image)
    feature = feature.reshape(1, 48, 48, 1)
    return feature / 255.0

# Open the webcam (camera)
try:
  webcam = cv2.VideoCapture(0)
except cv2.VideoCaptureError as e:
  print("Error opening webcam:", e)
  exit()


# Define labels for emotion classes
labels = {0: 'angry', 1: 'disgust', 2: 'fear', 3: 'happy', 4: 'neutral', 5: 'sad', 6: 'surprise'}

while True:
    try:
       i, im = webcam.read() #READ the webcam (camera)
    except cv2.VideoCaptureError as e:
      print("Error reading webcam frame:", e)
      exit()
    # Continue the loop (optional) or break and exit
    continue  # This will continue to the next iteration
#i, im = webcam.read()

    # Convert the frame to grayscale
    gray = cv2.cvtColor(im, cv2.COLOR_BGR2GRAY)

    # Detect faces in the grayscale frame
    faces = face_cascade.detectMultiScale(im, 1.3, 5)

    try:
        # For each detected face, perform facial emotion recognition
        for (p, q, r, s) in faces:
            # Extract the region of interest (ROI) which contains the face
            image = gray[q:q + s, p:p + r]

            # Draw a rectangle around the detected face
            cv2.rectangle(im, (p, q), (p + r, q + s), (255, 0, 0), 2)

            # Resize the face image to the required input size (48x48)
            image = cv2.resize(image, (48, 48))

            # Extract features from the resized face image
            img = extract_features(image)

            # Make a prediction using the trained model
            pred = model.predict(img)

            # Get the predicted label for emotion
            prediction_label = labels[pred.argmax()]

            # Display the predicted emotion label near the detected face
            cv2.putText(im, f'Emotion: {prediction_label}', (p - 10, q - 10),
                        cv2.FONT_HERSHEY_COMPLEX_SMALL, 2, (0, 0, 255))

        # Display the frame with annotations in real-time
        cv2.imshow("Real-time Facial Emotion Recognition", im)

        # Break the loop if the 'Esc' key is pressed
        if cv2.waitKey(1) == 27:
            break

    except cv2.error:
        pass

# Release the webcam and close all OpenCV windows
webcam.release()
cv2.destroyAllWindows()